In [ ]:
import osiris_utils as ou
import numpy as np
from pathlib import Path
from tqdm import tqdm
import h5py
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.size'] = 14

In [ ]:

path_init_field_fixed = "/home/exxxx5/Tese/Decks/weibelTestsFinalV2/TestElecsFrozenWeibel/weibelBG_B0_1.in"
sim_init_field_fixed = ou.Simulation(path_init_field_fixed)
sim_init_field_fixed["test_electrons"]["tracks"].load_all()

path_full_sim = "/home/exxxx5/Tese/Decks/weibelTestsFinalV2/TestElecsWithWeibel/weibelBG_B0_1.in"
sim_full_sim = ou.Simulation(path_full_sim)
sim_full_sim["test_electrons"]["tracks"].load_all()

In [ ]:
def plot_hist(track, t=-1, fig=None, ax=None):
    if fig is None or ax is None:
        fig, ax = plt.subplots()
    
    label = ("$t = {:.2f}$".format(track["t"][0, t]) + "$[{}]$".format(track.units["t"]))

    p = np.sqrt(track["p1"][:, t]**2 + track["p2"][:, t]**2 + track["p3"][:, t]**2)
    ax.hist(p, bins=50, label=label,
            histtype='step', linewidth=1.5)  # <- only contour

    ax.set_xlabel("${}$".format("p") + "$[{}]$".format(track.units["p3"]))
    plt.legend()
    return fig, ax


In [ ]:
def plot_hist(track, t=-1, fig=None, ax=None, nbins=50, logy=False, label=None):
    if fig is None or ax is None:
        fig, ax = plt.subplots()

    if label is None:
        label = (r"$t = {:.2f}$".format(track["t"][0, t]) +
                r"$[{}]$".format(track.units["t"]))

    # total momentum magnitude
    p = np.sqrt(track["p1"][:, t]**2 + track["p2"][:, t]**2 + track["p3"][:, t]**2)

    # keep only positive values for log bins
    p_pos = p[p > 0]
    if p_pos.size == 0:
        raise ValueError("All p values are non-positive; cannot make log-spaced bins.")

    # log-spaced bins between min and max of positive values
    bins = np.logspace(np.log10(p_pos.min()), np.log10(p_pos.max()), nbins + 1)

    ax.hist(p_pos, bins=bins, label=label, histtype='step', linewidth=1.5)

    ax.set_xscale('log')                 # log-spacing on x-axis
    if logy:
        ax.set_yscale('log')            # optional: log counts

    ax.set_xlabel(r"$u$" + r"$[c]$")
    ax.set_ylabel("Counts")
    ax.legend()
    return fig, ax


In [ ]:
fig, ax = None, None

# fig, ax = plot_hist(sim_full_sim["test_electrons"]["tracks"], t=-37, fig=fig, ax=ax)

fig, ax = plot_hist(sim_init_field_fixed["test_electrons"]["tracks"], t=0, fig=fig, ax=ax, label="Initial Distribution")

fig, ax = plot_hist(sim_full_sim["test_electrons"]["tracks"], t=-1, fig=fig, ax=ax, label="Consistent Background")


fig, ax = plot_hist(sim_init_field_fixed["test_electrons"]["tracks"], t=-1, fig=fig, ax=ax, label="Frozen Background")

plt.tight_layout()
plt.show()
